In [118]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [119]:
df = pd.read_parquet("/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_processed.parquet")
# df = pd.read_parquet("/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_small_for_analysis.parquet")

In [ ]:
df.info()

In [117]:
df.head()

,id,title,url,subreddit,created_utc,parsed_utc,text,score,num_comments,type,...,clean_text,processed_text,processed_text_length,sentiment_scores,entities,doc_embedding,tickers,ticker,created_price,price_1d
0,m0miqte,None,None,investing,2024-12-05 23:58:30,2024-12-06 17:55:40,Depends on what type of debt. For example if i...,3,NaN,comment,...,depends type debt. for example it’s debt mortg...,depend type debt for exampl it debt mortgag at...,594,-0.0772,"[PATH, FDIS, RPA, BTI, AMEX, Robotic Process A...","[1.0175916, -0.26974982, -0.18348941, -0.53738...","[PATH, BTI]",PATH,14.940000,14.820000
1,m0lmgzo,None,None,investing,2024-12-05 21:00:41,2024-12-06 17:55:40,KAR and PRAA,1,NaN,comment,...,kar praa,kar praa,12,0.0000,[KAR],"[0.0036067592, -0.009290272, -0.0037351453, -0...",[KAR],KAR,20.250000,20.190001
2,m0lvb3h,None,None,investing,2024-12-05 21:46:35,2024-12-06 17:55:41,"36 years old, married but no kids yet. Have 90...",4,NaN,comment,...,"36 years old, married kids yet. have 90% equit...",year old marri kid yet have equiti short term ...,433,-0.6652,"[ETH, VGT, QQQ, VOO]","[0.24014705, -0.79617554, -0.21952175, 0.06111...","[ETH, QQQ]",QQQ,521.799988,526.510010
3,1h7i31g,My biggest fear for 2025 is declines becoming ...,https://www.reddit.com/r/investing/comments/1h...,investing,2024-12-05 19:53:10,2024-12-06 17:55:42,"When has this not been a risk, it's why the ec...",102,92.0,submission,...,"when risk, economy acts ratchet, going slow fa...",when risk economi act ratchet go slow fast but...,1362,-0.9131,"[Economics, fed, AI]","[0.36487904, -0.9539981, -0.13656119, -1.58051...",[AI],AI,37.490002,40.520000
4,m0m473e,None,None,investing,2024-12-05 22:33:54,2024-12-06 17:55:42,I'm waiting for Tesla to pop. From there I spe...,1,NaN,comment,...,i'm waiting tesla pop. from i speculate automa...,i wait tesla pop from i specul automak tech bu...,332,-0.1280,"[Tesla, AI]","[0.4363078, -0.67959464, 0.8144916, 0.16138163...",[AI],AI,37.490002,40.520000


### Remove empty entities

In [3]:
df_ent = df[df['entities'].apply(lambda x: len(x) != 0)]

In [4]:
df_ent['id'].nunique()

36768

In [6]:
exploded = df_ent['entities'].explode()
exploded.value_counts()

,count
entities,
NVDA,1632
AI,1488
ACHR,1226
MSTR,1210
SPY,1058
...,...
Rheinmetal,1
CLS Celestica Inc.,1
Eton,1


### Filter entities by ticker list

In [7]:
import requests

url = "https://www.sec.gov/files/company_tickers.json"
headers = {
    "User-Agent": "john doe johndoe@gmail.com"
}

response = requests.get(url, headers=headers)
response.raise_for_status()
data = response.json()

sec_tickers = set()

for entry in data.values():
    sec_tickers.add(entry['ticker'].upper())

In [11]:
len(sec_tickers)

9999

In [9]:
known_tickers = set()
known_tickers.add('^GSPC')

In [10]:
sec_tickers.update(known_tickers)

In [ ]:
# do this during NER phase
df_ent['entities'] = df_ent['entities'].apply(lambda x: ["^GSPC" if e == "S&P" else e for e in x])

In [ ]:
df_ent['tickers'] = df_ent['entities'].apply(lambda x: [e.upper() for e in x if e.upper() in sec_tickers])

In [ ]:
# df_ent[df_ent['id'] == 'ltr5fuy'][['entities', 'tickers']]

,entities,tickers
9,"[QQQ, Invesco, Vanguard Megacaps Growth, VOO, MGK, ^GSPC]","[QQQ, ^GSPC]"


In [14]:
df_tick = df_ent[df_ent['tickers'].apply(lambda x: len(x) != 0)]

In [16]:
df_tick[['id', 'entities', 'tickers']].head(10)

,id,entities,tickers
0,1h85vjq,"[Mag 7, NVDA, Trump]",[NVDA]
1,1h85ju3,"[BYND - Key Data Points, BYND, Short Interest]",[BYND]
4,m0q4cv2,[SPY],[SPY]
14,1h841ha,[AI],[AI]
27,1h81owq,"[Drop, AI, ARR, EPS]","[DROP, AI, ARR]"
28,1h81k4n,"[UNH, mac & cheese]",[UNH]
35,m0pexsj,[UNH],[UNH]
36,m0pg8ej,[UNH],[UNH]
43,m0pgutx,[UNH],[UNH]
53,m0pswpc,"[TD, Treasury Direct]",[TD]


### Select random ticker from tickers as 'main' ticker (could be improved by selecting the mode, but it requires dataset adjustments)

In [17]:
import random
df_tick['ticker'] = df_tick['tickers'].apply(lambda x: random.choice(x))

<ipython-input-17-34810f28dd0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tick['ticker'] = df_tick['tickers'].apply(lambda x: random.choice(x))


In [18]:
df_tick[['created_utc', 'id', 'entities', 'tickers', 'ticker']].head(10)

,created_utc,id,entities,tickers,ticker
0,2024-12-06 16:54:17,1h85vjq,"[Mag 7, NVDA, Trump]",[NVDA],NVDA
1,2024-12-06 16:40:25,1h85ju3,"[BYND - Key Data Points, BYND, Short Interest]",[BYND],BYND
4,2024-12-06 16:10:38,m0q4cv2,[SPY],[SPY],SPY
14,2024-12-06 15:35:27,1h841ha,[AI],[AI],AI
27,2024-12-06 13:46:26,1h81owq,"[Drop, AI, ARR, EPS]","[DROP, AI, ARR]",ARR
28,2024-12-06 13:39:35,1h81k4n,"[UNH, mac & cheese]",[UNH],UNH
35,2024-12-06 13:43:24,m0pexsj,[UNH],[UNH],UNH
36,2024-12-06 13:51:58,m0pg8ej,[UNH],[UNH],UNH
43,2024-12-06 13:56:02,m0pgutx,[UNH],[UNH],UNH
53,2024-12-06 15:08:46,m0pswpc,"[TD, Treasury Direct]",[TD],TD


### Add prices (do it at least 2 days after parsing)

In [ ]:
!pip install yfinance

In [ ]:
df_tick['created_utc'] = pd.to_datetime(df_tick['created_utc'])
df_tick = df_tick.reset_index(drop=True)

### Debug single price

In [99]:
import yfinance as yf

date = pd.to_datetime('2024-10-25').date()

# stock = yf.Ticker('^GSPC')
# hist = stock.history(start=str(date), end=str(date + pd.Timedelta(days=1)), interval = "30m")
data = yf.download("CAT", start="2024-11-21", end="2024-11-22",  interval = "30m")

[*********************100%***********************]  1 of 1 completed


In [103]:
data.iloc[12]['Close', 'CAT']

389.6099853515625

In [140]:
import time
import yfinance as yf

def get_ticker_price(ticker, target_time, price_type='Close', interval='30m', sleep_seconds=0):
  date = target_time.date()
  prices = yf.download(ticker, start=str(date), end=str(date + pd.Timedelta(days=1)), interval=interval, progress=False)

  if prices.empty:
    return 0

  prices['time_diff'] = abs(prices.index - target_time)
  closest_time = prices['time_diff'].idxmin()
  closest_row = prices.loc[closest_time]

  time.sleep(sleep_seconds)

  try:
    return closest_row[price_type, ticker]
  except KeyError:
    print("KeyError occurred, returning 0")
    return 0

In [ ]:
df[df['ticker'] == 'SAVE'][['created_utc', 'ticker', 'created_price']]

In [45]:
df_tick.shape

(16426, 20)

In [29]:
df_tick.index

RangeIndex(start=0, stop=16426, step=1)

In [26]:
df_tick = df_tick.reset_index(drop=True)

In [148]:
df_2nd_stage.iloc[900:1000]['price_1d']

KeyError: 'price_1d'

### This needs to be restarted if breaks

In [143]:
import yfinance as yf

start_index = 14900

for i in range(start_index, df.shape[0], 100):
    print(i)
    df_tick_slice = df.iloc[i:i+100].copy()

    # df_tick_slice['created_price'] = df_tick_slice.apply(lambda x: get_ticker_price(x['ticker'], x['created_utc']), axis=1)
    df_tick_slice['created_price'] = df_tick_slice.apply(lambda x: get_ticker_price(x['ticker'], x['created_utc']) if pd.isnull(x['created_price']) else x['created_price'], axis=1)

    # Update the original DataFrame
    df.loc[df_tick_slice.index, 'created_price'] = df_tick_slice['created_price']

    # Save to parquet after each batch
    df.to_parquet('/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_processed.parquet', index=False)

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PLTR']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NVDA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZIM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RJF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NIO']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['QQQ']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yf

14900


ERROR:yfinance:['ULTA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MSTR']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GE']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ASTS']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TSM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['QQQ']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yf

15000


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ASML']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GTBIF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RCAT']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['WBA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PEP']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NMFC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-23 -> 2024-11-24)')
ERROR

...

KeyError occurred, returning 0
KeyError occurred, returning 0
KeyError occurred, returning 0
KeyError occurred, returning 0
KeyError occurred, returning 0
KeyError occurred, returning 0


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['LRCX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-03 -> 2024-11-04)')


KeyError occurred, returning 0
KeyError occurred, returning 0
KeyError occurred, returning 0
16300


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['LEU']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CCO']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-03 -> 2024-11-04)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PIF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-10-31 -> 2024-11-01)')


16400


In [ ]:
df.to_parquet('/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_processed.parquet')

In [144]:
df['created_price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 16426 entries, 0 to 16425
Series name: created_price
Non-Null Count  Dtype  
--------------  -----  
16330 non-null  float64
dtypes: float64(1)
memory usage: 128.5 KB


### Add +1d price

In [145]:
df_2nd_stage = df[df['created_price'].notnull() & (df['created_price'] != 0)]

In [146]:
df_2nd_stage.shape

(11625, 20)

In [149]:
import yfinance as yf

start_index = 0

for i in range(start_index, df_2nd_stage.shape[0], 100):
    print(i)
    df_tick_slice = df_2nd_stage.iloc[i:i+100].copy()

    # df_tick_slice['created_price'] = df_tick_slice.apply(lambda x: get_ticker_price(x['ticker'], x['created_utc']), axis=1)
    df_tick_slice['price_1d'] = df_tick_slice.apply(lambda x: get_ticker_price(x['ticker'], x['created_utc'] + pd.Timedelta(days=1)), axis=1)

    # Update the original DataFrame
    df_2nd_stage.loc[df_tick_slice.index, 'price_1d'] = df_tick_slice['price_1d']

    # Save to parquet after each batch
    df_2nd_stage.to_parquet('/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_prices.parquet', index=False)

0


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NVDA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BYND']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPY']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AI']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ARR']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['UNH']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfi

100


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPY']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SMCI']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-12-07 -> 2024-12-08)')
E

200


...

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FCF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['OXY']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['OXY']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IBM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DCF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfina

11500


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ACIC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['UAL']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DEO']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBSW']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ASPN']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SBC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')
ERROR:y

11600


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['META']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (15m 2024-11-02 -> 2024-11-03)')


In [154]:
df_2nd_stage.shape

(8416, 21)

In [151]:
df_2nd_stage = df_2nd_stage[df_2nd_stage['price_1d'].notnull() & (df_2nd_stage['price_1d'] != 0)]

In [155]:
df_2nd_stage.to_parquet('/content/drive/MyDrive/datasets/reddit_parser_2024_12_06_prices.parquet', index=False)